In [ ]:
# !pip install gensim

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Markdown
import ast
from scipy import sparse
import matplotlib.pyplot as plt

import re
import nltk #Natural Language Toolkit
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
import string
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from joblib import dump, load
import pickle


In [3]:
tmdb_5000_movies_df   = pd.read_csv('/content/tmdb_5000_movies.csv', engine='python', on_bad_lines='warn')
tmdb_5000_credits_df  = pd.read_csv('/content/tmdb_5000_credits.csv', engine='python', on_bad_lines='warn')

def Explore(title_, dataframe_):
    display(Markdown(title_))
    display(Markdown('---'))
    display(dataframe_.tail())
    display('Descriptive Statistics')
    display(dataframe_.describe())
    display('Missing Values')
    display(dataframe_.isnull().sum().sort_values(ascending=False))
    display('Column Data Types')
    display(dataframe_.dtypes)

Explore('### Movies Data', tmdb_5000_movies_df)
Explore('### Credits Data', tmdb_5000_credits_df)

### Movies Data

---

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6
4801,0,[],http://shanghaicalling.com/,126186,[],en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
4802,0,"[{""id"": 99, ""name"": ""Documentary""}]",NaN,25975,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 224...",en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,My Date with Drew,6.3,16


'Descriptive Statistics'

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


...

,movie_id
count,4803.000000
mean,57165.484281
std,88694.614033
min,5.000000
25%,9014.500000
50%,14629.000000
75%,58610.500000
max,459488.000000


'Missing Values'

,0
movie_id,0
title,0
cast,0
crew,0


'Column Data Types'

,0
movie_id,int64
title,object
cast,object
crew,object


# Remarks
## Movies Dataframe
### Object Columns
1. Columns genres and keywords are object columns that contains dictionaries bounded within lists i.e. [{'id' : id, 'name' : genre/keyword}] a film can have multiple genres or keywords.
2. homepage contains web adrresses for some of the movies it also has the most missing row values (3091/4803), over 60%.
3. production_companies is an object column that contains dictionaries bounded within lists i.e. [{'name' : production_companies, 'id' : id}] a film can have multiple production_companies.
4. production_countries is an object column that contains dictionaries bounded within lists i.e. [{'iso code' : code letters e.g US, 'name' : production_countries}] a film can have multiple production_countries.
5. release_date is an object column that output the date of release, a good feature for filtering year of release.
6. spoken_languages is an object columns that contains dictionaries bounded within lists i.e. [{'iso code' : code letters e.g US, 'name' : spoken_languages}] a film can have multiple spoken_languages.
7. status is an object column that has 3 unique values i.e ['Released' 'Post Production' 'Rumored']
8. tagline stores individual movie. A tagline is a reiterated phrase identified with an individual, group, or product : slogan. The column also has the second highest missing row values (844/4803), about 18%
9. Title an object columns most likely represents the final title awarded to the movie as welll as original_language and original_title representing the original language and original title
10. overview is the description column for the movie where a brief idea of the movie is illustrated. It has only 3 missing row values (3/4803) only 0.06%

### Float/Int Columns
1. Budget simply stores the movies budget.
2. Popularity stored the popularity ranging from 0 to 876 according to the above dataset.
3. Revenue is simply how much the movie made back.
4. Runtime is movie length in minutes and only has 2 missing row values (2/4803) only 0.04%
5. Vote Average is probably a standardize vote from 0 to 10.
6. Vote Count is the number of people who voted per movie.

## Credits Dataframe
1. Cast is an object columns that contains dictionaries bounded within lists i.e. [{'cast id' : cast_id, 'character' : character's_name, 'credit_id' : unique_float, 'gender' : 0, 'id' : float, 'name' : actors_name, 'order' : int}]
2. Crew is an object column that contains dictionaries bounded within lists i.e. [{"credit_id": "553bef6a9251416874003c8f", "department": "Production", "gender": 2, "id": 3276, "job": "Casting", "name": "Roger Mussenden"}]
3. Movie Id and Title are columns used for reference with the first dataframe.

---
# Data Cleaning

In [4]:
'''
Dropping Homepage and Tagline because they have a significant amount of missing values
Homepage contains web addresses so it may not be as useful.
Tagline is a short quote like statement based on the film, similar information may be present in the overview column.
Will test more in future models.
'''
tmdb_5000_movies_df.drop(['homepage', 'tagline'], axis=1, inplace=True)

In [5]:
'''
Imputing rows with missing data in overview, runtime and release_date since only a small
fraction of the entire column is missing i.e. 0.06%, 0/04% and 0.02% respectively.
'''
tmdb_5000_movies_df.dropna(subset=['overview', 'runtime', 'release_date'], inplace=True)

In [6]:
# Confirm missing values
tmdb_5000_movies_df.isnull().sum()

,0
budget,0
genres,0
id,0
keywords,0
original_language,0
original_title,0
overview,0
popularity,0
production_companies,0
production_countries,0


---
# Feature Engineering

### Merging tmdb_5000_movies_df and tmdb_5000_credits_df to create cast and crew feature

In [7]:
tmdb_5000_credits_df.rename(columns={"movie_id": "id"}, inplace=True)
tmdb_5000_movies_df.rename(columns={"original_title": "title"}, inplace=True)
print(tmdb_5000_credits_df.columns)
print(tmdb_5000_movies_df.columns)

Index(['id', 'title', 'cast', 'crew'], dtype='object')
Index(['budget', 'genres', 'id', 'keywords', 'original_language', 'title',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title', 'vote_average', 'vote_count'],
      dtype='object')


In [8]:
tmdb_5000_merged_df = pd.merge(tmdb_5000_movies_df, tmdb_5000_credits_df, how='inner', on='id')
tmdb_5000_merged_df.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language', 'title_x',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'title_x', 'vote_average', 'vote_count',
       'title_y', 'cast', 'crew'],
      dtype='object')

In [9]:
print(tmdb_5000_merged_df['crew'][0])
print(tmdb_5000_merged_df['crew'].dtype)

[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"}, 

### Creating new features; genres_string, keywords_string, cast_string, crew_string, production_company_string and  production_country_string

In [10]:
extraccted_columns = ['keywords', 'genres','cast', 'crew', 'production_companies', 'production_countries']

# Converting object list of dictionaries to actual lists of dictionaries
for col in extraccted_columns:
  tmdb_5000_merged_df[col] = tmdb_5000_merged_df[col].apply(ast.literal_eval)

def extract_names(data_list, key):
    return ', '.join([d.get(key, '') for d in data_list if key in d])

# Map of new column name to (source column, key in dict)
extraction_map = {
    'genres_string': ('genres', 'name'),
    'keywords_string': ('keywords', 'name'),
    'actor_string': ('cast', 'name'),
    'cast_string': ('cast', 'character'),
    'crew_name_string': ('crew', 'name'),
    'crew_department_string': ('crew', 'department'),
    'production_company_string': ('production_companies', 'name'),
    'production_country_string': ('production_countries', 'iso_3166_1')
}

# Apply extraction
for new_col, (source_col, key) in extraction_map.items():
    tmdb_5000_merged_df[new_col] = tmdb_5000_merged_df[source_col].apply(lambda x: extract_names(x, key))

# Converting release_date to a datetime object and acessing year
tmdb_5000_merged_df['release_date'] = pd.to_datetime(tmdb_5000_merged_df['release_date'])
tmdb_5000_merged_df['year'] = tmdb_5000_merged_df['release_date'].dt.year

tmdb_5000_merged_df.head()

,budget,genres,id,keywords,original_language,title_x,overview,popularity,production_companies,production_countries,...,crew,genres_string,keywords_string,actor_string,cast_string,crew_name_string,crew_department_string,production_company_string,production_country_string,year
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe48009251416c750aca23', 'de...","Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","Sam Worthington, Zoe Saldana, Sigourney Weaver...","Jake Sully, Neytiri, Dr. Grace Augustine, Col....","Stephen E. Rivkin, Rick Carter, Christopher Bo...","Editing, Art, Sound, Sound, Production, Sound,...","Ingenious Film Partners, Twentieth Century Fox...","US, GB",2009
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe4232c3a36847f800b579', 'de...","Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Johnny Depp, Orlando Bloom, Keira Knightley, S...","Captain Jack Sparrow, Will Turner, Elizabeth S...","Dariusz Wolski, Gore Verbinski, Jerry Bruckhei...","Camera, Directing, Production, Writing, Writin...","Walt Disney Pictures, Jerry Bruckheimer Films,...",US,2007
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",...,"[{'credit_id': '54805967c3a36829b5002c41', 'de...","Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...","Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...","James Bond, Blofeld, Madeleine, M, Lucia, Q, M...","Thomas Newman, Sam Mendes, Anna Pinnock, John ...","Sound, Directing, Art, Writing, Writing, Art, ...","Columbia Pictures, Danjaq, B24","GB, US",2015
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...","Christian Bale, Michael Caine, Gary Oldman, An...","Bruce Wayne / Batman, Alfred Pennyworth, James...","Hans Zimmer, Charles Roven, Christopher Nolan,...","Sound, Production, Writing, Directing, Product...","Legendary Pictures, Warner Bros., DC Entertain...",US,2012
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...","Taylor Kitsch, Lynn Collins, Samantha Morton, ...","John Carter, Dejah Thoris, Sola, Tars Tarkas, ...","Andrew Stanton, Andrew Stanton, John Lasseter,...","Writing, Directing, Crew, Production, Producti...",Walt Disney Pictures,US,2012


In [11]:
# Columns to transform.
columns_to_lower = ['overview']
# columns_to_lower = ['overview', 'genres_string','keywords_string', 'cast_string', 'actor_string', 'production_company_string', 'production_country_string']

# Convert training data samples to lowercase and using regex to remove special characters.
# 'map' applies the function to individual cells.
for col in columns_to_lower:
    tmdb_5000_merged_df[col] = tmdb_5000_merged_df[col].map(lambda x: x.lower() if isinstance(x, str) else x)
    tmdb_5000_merged_df[col] = tmdb_5000_merged_df[col].map(lambda x: re.sub(r'[^a-zA-Z\s]', ' ', x))

tmdb_5000_merged_df.head()

,budget,genres,id,keywords,original_language,title_x,overview,popularity,production_companies,production_countries,...,crew,genres_string,keywords_string,actor_string,cast_string,crew_name_string,crew_department_string,production_company_string,production_country_string,year
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,in the nd century a paraplegic marine is di...,150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe48009251416c750aca23', 'de...","Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","Sam Worthington, Zoe Saldana, Sigourney Weaver...","Jake Sully, Neytiri, Dr. Grace Augustine, Col....","Stephen E. Rivkin, Rick Carter, Christopher Bo...","Editing, Art, Sound, Sound, Production, Sound,...","Ingenious Film Partners, Twentieth Century Fox...","US, GB",2009
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead ha...,139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe4232c3a36847f800b579', 'de...","Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Johnny Depp, Orlando Bloom, Keira Knightley, S...","Captain Jack Sparrow, Will Turner, Elizabeth S...","Dariusz Wolski, Gore Verbinski, Jerry Bruckhei...","Camera, Directing, Production, Writing, Writin...","Walt Disney Pictures, Jerry Bruckheimer Films,...",US,2007
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,a cryptic message from bond s past sends him o...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",...,"[{'credit_id': '54805967c3a36829b5002c41', 'de...","Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...","Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...","James Bond, Blofeld, Madeleine, M, Lucia, Q, M...","Thomas Newman, Sam Mendes, Anna Pinnock, John ...","Sound, Directing, Art, Writing, Writing, Art, ...","Columbia Pictures, Danjaq, B24","GB, US",2015
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,following the death of district attorney harve...,112.312950,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...","Christian Bale, Michael Caine, Gary Oldman, An...","Bruce Wayne / Batman, Alfred Pennyworth, James...","Hans Zimmer, Charles Roven, Christopher Nolan,...","Sound, Production, Writing, Directing, Product...","Legendary Pictures, Warner Bros., DC Entertain...",US,2012
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,john carter is a war weary former military ca...,43.926995,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...","Taylor Kitsch, Lynn Collins, Samantha Morton, ...","John Carter, Dejah Thoris, Sola, Tars Tarkas, ...","Andrew Stanton, Andrew Stanton, John Lasseter,...","Writing, Directing, Crew, Production, Producti...",Walt Disney Pictures,US,2012


In [12]:
# Download stopwords onto ram
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

tmdb_5000_merged_df['overview'] = tmdb_5000_merged_df['overview'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
tmdb_5000_merged_df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,budget,genres,id,keywords,original_language,title_x,overview,popularity,production_companies,production_countries,...,crew,genres_string,keywords_string,actor_string,cast_string,crew_name_string,crew_department_string,production_company_string,production_country_string,year
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,nd century paraplegic marine dispatched moon p...,150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe48009251416c750aca23', 'de...","Action, Adventure, Fantasy, Science Fiction","culture clash, future, space war, space colony...","Sam Worthington, Zoe Saldana, Sigourney Weaver...","Jake Sully, Neytiri, Dr. Grace Augustine, Col....","Stephen E. Rivkin, Rick Carter, Christopher Bo...","Editing, Art, Sound, Sound, Production, Sound,...","Ingenious Film Partners, Twentieth Century Fox...","US, GB",2009
1,300000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",285,"[{'id': 270, 'name': 'ocean'}, {'id': 726, 'na...",en,Pirates of the Caribbean: At World's End,captain barbossa long believed dead come back ...,139.082615,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe4232c3a36847f800b579', 'de...","Adventure, Fantasy, Action","ocean, drug abuse, exotic island, east india t...","Johnny Depp, Orlando Bloom, Keira Knightley, S...","Captain Jack Sparrow, Will Turner, Elizabeth S...","Dariusz Wolski, Gore Verbinski, Jerry Bruckhei...","Camera, Directing, Production, Writing, Writin...","Walt Disney Pictures, Jerry Bruckheimer Films,...",US,2007
2,245000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",206647,"[{'id': 470, 'name': 'spy'}, {'id': 818, 'name...",en,Spectre,cryptic message bond past sends trail uncover ...,107.376788,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",...,"[{'credit_id': '54805967c3a36829b5002c41', 'de...","Action, Adventure, Crime","spy, based on novel, secret agent, sequel, mi6...","Daniel Craig, Christoph Waltz, Léa Seydoux, Ra...","James Bond, Blofeld, Madeleine, M, Lucia, Q, M...","Thomas Newman, Sam Mendes, Anna Pinnock, John ...","Sound, Directing, Art, Writing, Writing, Art, ...","Columbia Pictures, Danjaq, B24","GB, US",2015
3,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",49026,"[{'id': 849, 'name': 'dc comics'}, {'id': 853,...",en,The Dark Knight Rises,following death district attorney harvey dent ...,112.312950,"[{'name': 'Legendary Pictures', 'id': 923}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe4781c3a36847f81398c3', 'de...","Action, Crime, Drama, Thriller","dc comics, crime fighter, terrorist, secret id...","Christian Bale, Michael Caine, Gary Oldman, An...","Bruce Wayne / Batman, Alfred Pennyworth, James...","Hans Zimmer, Charles Roven, Christopher Nolan,...","Sound, Production, Writing, Directing, Product...","Legendary Pictures, Warner Bros., DC Entertain...",US,2012
4,260000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",49529,"[{'id': 818, 'name': 'based on novel'}, {'id':...",en,John Carter,john carter war weary former military captain ...,43.926995,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"[{'credit_id': '52fe479ac3a36847f813eaa3', 'de...","Action, Adventure, Science Fiction","based on novel, mars, medallion, space travel,...","Taylor Kitsch, Lynn Collins, Samantha Morton, ...","John Carter, Dejah Thoris, Sola, Tars Tarkas, ...","Andrew Stanton, Andrew Stanton, John Lasseter,...","Writing, Directing, Crew, Production, Producti...",Walt Disney Pictures,US,2012


---
# Vectorizing and Standardizing

$$
TF(w) = \frac{\text{Number of times w appears in d}}{\text{Total number of words in d}}
$$
where :
- w is word
- d is document(sample)
$$
IDF(w) = \log({\frac{N}{\text{DF}(w)}+1})
$$
where :
- N = Total number of documents(samples)
- DF(w) = Number of documents that contain w
- Adding 1 in the denominator prevents division by zero

$$
TFIDF(w,d) = TF(w,d) * (\text{IDF}(w))
$$

- Implementation of the above formulas can differ based on how the TfidfVectorizer is intialized accounting for smoothing, normalization etc.

In [13]:
tmdb_5000_merged_df['combined_text'] = tmdb_5000_merged_df[['overview', 'genres_string', 'keywords_string', 'cast_string', 'actor_string', 'production_company_string', 'production_country_string']].apply(lambda x: ' '.join(x.dropna()), axis=1)
print(tmdb_5000_merged_df['combined_text'][1])

captain barbossa long believed dead come back life headed edge earth turner elizabeth swann nothing quite seems Adventure, Fantasy, Action ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, shipwreck, strong woman, ship, alliance, calypso, afterlife, fighter, pirate, swashbuckler, aftercreditsstinger Captain Jack Sparrow, Will Turner, Elizabeth Swann, William "Bootstrap Bill" Turner, Captain Sao Feng, Captain Davy Jones, Captain Hector Barbossa, Admiral James Norrington, Joshamee Gibbs, Lord Cutler Beckett, Tia Dalma, Governor Weatherby Swann, Captain Teague Sparrow, Pintel, Ragetti, Lieutenant Theodore Groves, Cotton, Marty, Ian Mercer, Scarlett, Giselle, Mullroy, Murtogg, Tai Huang, Henry Turner, Mistress Ching, Lieutenant Greitzer, Hadras, Clacker, Penrod, Cotton's Parrot (voice), Captain Teague, Captain Jocard, Captain Ammand Johnny Depp, Orlando Bloom, Keira Knightley, Stellan Skarsgård, Chow Yun-fat, Bill Nighy, Geoffrey Rush, Jack Davenpor

In [14]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tmdb_5000_merged_df['combined_text'])

In [15]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# 1. Text Preprocessing Function
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Lowercase & tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english') and word not in string.punctuation]
    return tokens

In [17]:
tmdb_5000_merged_df['tokens'] = tmdb_5000_merged_df[['overview', 'genres_string', 'keywords_string', 'cast_string', 'actor_string', 'production_company_string'
  , 'production_country_string']].apply(lambda row: preprocess_text(' '.join(row.astype(str))), axis=1)

In [18]:
word2vec_model = Word2Vec(sentences=tmdb_5000_merged_df['tokens'], vector_size=300, window=5, min_count=1, sg=0)
word2vec_model.save("word2vec.model")

In [19]:
print(word2vec_model.wv.vectors.shape)

(75283, 300)


In [20]:
def get_movie_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

In [21]:
tmdb_5000_merged_df['vector'] = tmdb_5000_merged_df['tokens'].apply(lambda tokens: get_movie_vector(tokens, word2vec_model))

In [22]:
word2vec_matrix = np.vstack(tmdb_5000_merged_df['vector'].values)

In [23]:
tfidf_matrix.shape

(4799, 72603)

In [24]:
word2vec_matrix.shape

(4799, 300)

---
# Cosine Similarity

In [25]:
tmdb_5000_merged_df['vector'].shape

(4799,)

In [26]:
tmdb_5000_merged_df['combined_text'].shape

(4799,)

In [27]:
# converting the vector column in a 2D numpy array which is required for cosine similarity
movie_vectors = np.array([np.array(vec) for vec in tmdb_5000_merged_df['vector']])
movie_vectors.shape

(4799, 300)

In [28]:
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [29]:
def get_recommendations(target_title, similarities, df, top_n=10):
  # Check if the target title exists in the DataFrame
  matching_movies = df[df['title_y'].str.lower() == target_title.lower()]
  if matching_movies.empty:
    print(f"{target_title} not found in dataset.")
    return []

  idx = matching_movies.index[0]
  sim_scores = list(enumerate(similarities[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:top_n+1]
  movie_indices = [i[0] for i in sim_scores]
  return df['title_y'].iloc[movie_indices]

In [30]:
print(get_recommendations('bonker', similarities, tmdb_5000_merged_df))

bonker not found in dataset.
[]


In [31]:
with open('data_and_similarities.pkl', 'wb') as f:
  pickle.dump((tmdb_5000_merged_df, similarities), f)